In [2]:
import numpy as np
import pandas as pd
import sys
import gymnasium as gym
import gym_trading_env

from ConvertCSV import DataReady 

In [5]:
df = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT5min.csv")

DataReady(df)

df = df[:-1000] # leave 1000 for testing